[Reference](https://medium.com/mindsdb/how-to-pick-the-best-performing-time-series-ai-model-for-your-specific-data-3480aae042da)

```
CREATE DATABASE mysql_demo_db
WITH ENGINE = "mysql",
PARAMETERS = {
    "user": "user",
    "password": "MindsDBUser123!",
    "host": "db-demo-data.cwoyhfn6bzs0.us-east-1.rds.amazonaws.com",
    "port": "3306",
    "database": "public"
    };

SELECT *
FROM mysql_demo_db.historical_expenditures
LIMIT 10;


CREATE VIEW training_data (
   SELECT * FROM mysql_demo_db.historical_expenditures
   WHERE month NOT IN ('2016-10-01', '2016-11-01', '2016-12-01',
                       '2017-01-01', '2017-02-01', '2017-03-01',
                       '2017-04-01', '2017-05-01', '2017-06-01',
                       '2017-07-01', '2017-08-01', '2017-09-01')
);
```

# StatsForecast
```
CREATE ML_ENGINE statsforecast
FROM statsforecast;
CREATE MODEL statsforecast_model
FROM mindsdb
    (SELECT * FROM training_data)
PREDICT expenditure
ORDER BY month
GROUP BY category
WINDOW 120
HORIZON 12
USING ENGINE = 'statsforecast';
```

# NeuralForecast

```
CREATE ML_ENGINE neuralforecast
FROM neuralforecast;
CREATE MODEL neuralforecast_model
FROM mindsdb
    (SELECT * FROM training_data)
PREDICT expenditure
ORDER BY month
GROUP BY category
WINDOW 120
HORIZON 12
USING ENGINE = 'neuralforecast';
```

# TimeGPT

```
CREATE ML_ENGINE timegpt
FROM timegpt
USING
   timegpt_api_key = 'timegpt-api-key';

CREATE MODEL timegpt_model
FROM mindsdb
    (SELECT * FROM training_data)
PREDICT expenditure
ORDER BY month
GROUP BY category
HORIZON 12
USING ENGINE = 'timegpt';
```

# Forecasted vs. True Values


```
SELECT substring(month, 1, 10) AS month, category, expenditure
FROM mysql_demo_db.historical_expenditures
WHERE category = 'industry'
AND (month = '2016-10-01' OR month = '2016-11-01' OR month = '2016-12-01'
     OR month = '2017-01-01' OR month = '2017-02-01' OR month = '2017-03-01'
     OR month = '2017-04-01' OR month = '2017-05-01' OR month = '2017-06-01'
     OR month = '2017-07-01' OR month = '2017-08-01' OR month = '2017-09-01');
```

```
SELECT substring(m.month, 1, 10) AS month, m.category, m.expenditure
FROM training_data AS d
JOIN statsforecast_model AS m
WHERE d.category = 'industry'
AND d.month > LATEST
LIMIT 12;
```

```
SELECT substring(m.month, 1, 10) AS month, m.category, m.expenditure
FROM training_data AS d
JOIN neuralforecast_model AS m
WHERE d.category = 'industry'
AND d.month > LATEST
LIMIT 12;
```

```
SELECT substring(m.month, 1, 10) AS month, m.category, m.expenditure
FROM training_data AS d
JOIN timegpt_model AS m
WHERE d.category = 'industry'
AND d.month > LATEST
LIMIT 12;
```

# Performance Metrics to Evaluate Time-Series Models

```
EVALUATE mean_absolute_error
FROM (SELECT column_name_that_stores_real_value AS actual,
             column_name_that_stores_predicted_value AS prediction
      FROM table);
```

# Real-Time Forecasts with MindsDB

```
CREATE JOB get_real_time_forecasts (


    -- retraining models using the latest historical data
    RETRAIN statsforecast_model
    FROM mysql_demo_db
        (SELECT * FROM historical_expenditures)
    USING
        join_learn_process = true;


    RETRAIN neuralforecast_model
    FROM mysql_demo_db
        (SELECT * FROM historical_expenditures)
    USING
        join_learn_process = true;


    RETRAIN timegpt_model
    FROM mysql_demo_db
        (SELECT * FROM historical_expenditures)
    USING
        join_learn_process = true;


    -- sending forecasts to slack
    -- how to connect slack to mindsdb: https://docs.mindsdb.com/integrations/app-integrations/slack#method-2-chatbot-responds-on-a-defined-slack-channel
    INSERT INTO slack_app.channels (channel, text)
    VALUES("expenditure-forecasts", "Here are the expenditure forecasts for the next 12 months made by StatsForecast:");


    INSERT INTO slack_app.channels (channel, text)
        SELECT "expenditure-forecasts" AS channel,
               concat(m.month, ' --> ', m.expenditure) AS text
        FROM mysql_demo_db.historical_expenditures
        JOIN statsforecast_model AS m
        WHERE d.category = 'industry'
        AND d.month > LATEST
        LIMIT 12;


    INSERT INTO slack_app.channels (channel, text)
    VALUES("expenditure-forecasts", "Here are the expenditure forecasts for the next 12 months made by NeuralForecast:");


    INSERT INTO slack_app.channels (channel, text)
        SELECT "expenditure-forecasts" AS channel,
               concat(m.month, ' --> ', m.expenditure) AS text
        FROM mysql_demo_db.historical_expenditures
        JOIN neuralforecast_model AS m
        WHERE d.category = 'industry'
        AND d.month > LATEST
        LIMIT 12;


    INSERT INTO slack_app.channels (channel, text)
    VALUES("expenditure-forecasts", "Here are the expenditure forecasts for the next 12 months made by TimeGPT:");


    INSERT INTO slack_app.channels (channel, text)
        SELECT "expenditure-forecasts" AS channel,
               concat(m.month, ' --> ', m.expenditure) AS text
        FROM mysql_demo_db.historical_expenditures
        JOIN timegpt_model AS m
        WHERE d.category = 'industry'
        AND d.month > LATEST
        LIMIT 12;
)
EVERY 1 month;
```